In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv("/content/drive/MyDrive/KHUTHON/urls.csv", index_col = 0)
df.head()

,Title,thumbnail,text
0,Hadoop Yarn 아키텍쳐,https://velog.velcdn.com/images/matteblack9/po...,"개발Hadoop을 배우게 되면, MapReduce 다음으로 Yarn의 개념을 가장 ..."
1,하둡 프로그래밍 - MapReduce,https://velog.velcdn.com/images%2Fspdlqjfire%2...,"하둡하둡은 기본적으로 HDFS, MapReduce로 구성된다. 맵리듀스는 HDFS에..."
2,OpenSearch에 대해 알아보자!,https://velog.velcdn.com/images/sosimina/post/...,Opensearch는 Elastic Search가 기업화를 꾀하는데에 대한 반발로 ...
3,{일상} 10분만 일상,https://velog.velcdn.com/images/pairkorean/pro...,"{일상}10분동안 할게 뭐 있다고, 아둥바둥 힘들게 기록하나 싶다가도나한테 올바른 ..."
4,Airflow,https://images.velog.io/images/ifelifelse/post...,강의정리 - MLOps목차\nApache Airflow 소개\n-- Batch Pr...


## **키워드 추출**

In [36]:
!pip install keybert
!pip install sentence_transformers
!pip install konlpy

from keybert import KeyBERT
k_model = KeyBERT('distilbert-base-nli-mean-tokens')

import itertools
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# 한국어를 포함하고 있는 다국어 SBERT 를 로드
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [39]:
def keyword_func(doc, top_n, nr_candidates):
  # 형태소 분석기를 통해 명사만 추출한 문서 만들기
  okt = Okt()
  tokenized_doc = okt.pos(doc)
  tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

  # 사이킷런의 CountVectorizer를 사용해 단어 추출 / 이유: n_gram_range의 인자를 사용하면 쉽게 n-gram 추출이 가능하기 때문
  n_gram_range = (1, 2) # 결과 후보는 2 개의 단어를 한 묶음으로 간주하는 bigram 과 3 개의 단어를 한 묶음으로 간주하는 trigram 을 추출
  count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
  candidates = count.get_feature_names_out()
  doc_embedding = model.encode([doc]) # 문서 encoding하기
  candidate_embeddings = model.encode(candidates) # 추출된 단어 인코딩하기

  # 문서와 각 키워드들 간의 유사도
  distances = cosine_similarity(doc_embedding, candidate_embeddings)

  # 각 키워드들 간의 유사도
  distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)

  # 코사인유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
  words_idx = list(distances.argsort()[0][-nr_candidates:])
  words_vals = [candidates[index] for index in words_idx]
  distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

  # 각 키워드들중에서 가장 덜 유사한 키워드들 간의 조합을 계산 (# 상위 10 개의 키워드를 선택하고 이 10 개 중에서 서로 가장 유사성이 낮은 5 개를 선택)
  min_sim = np.inf

  candidate = None
  for combination in itertools.combinations(range(len(words_idx)), top_n):
    sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
    if sim < min_sim:
      candidate = combination
      min_sim = sim
  blog_keyword = [words_vals[idx] for idx in candidate]

  return blog_keyword

keyword_result = []
for txt in df['text']:
  keyword_result.append(keyword_func(txt, top_n=3, nr_candidates=24))

df['keyword'] = keyword_result

df.head()

,Title,thumbnail,text,keyword
0,Hadoop Yarn 아키텍쳐,https://velog.velcdn.com/images/matteblack9/po...,"개발Hadoop을 배우게 되면, MapReduce 다음으로 Yarn의 개념을 가장 ...","[스케줄링 로부터, 노드 매니저, 문제 본질]"
1,하둡 프로그래밍 - MapReduce,https://velog.velcdn.com/images%2Fspdlqjfire%2...,"하둡하둡은 기본적으로 HDFS, MapReduce로 구성된다. 맵리듀스는 HDFS에...","[파일 분산, 개발자 분석, 적용 알고리즘]"
2,OpenSearch에 대해 알아보자!,https://velog.velcdn.com/images/sosimina/post/...,Opensearch는 Elastic Search가 기업화를 꾀하는데에 대한 반발로 ...,"[라이센스 무료, 대시보드 라파, 검색 분석]"
3,{일상} 10분만 일상,https://velog.velcdn.com/images/pairkorean/pro...,"{일상}10분동안 할게 뭐 있다고, 아둥바둥 힘들게 기록하나 싶다가도나한테 올바른 ...","[생태계 구축, 일요일, 서든 착각]"
4,Airflow,https://images.velog.io/images/ifelifelse/post...,강의정리 - MLOps목차\nApache Airflow 소개\n-- Batch Pr...,"[소개 실습, 소프트웨어 자료구조, 에어비앤비 개발]"


## **문장요약**

In [40]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

def summarization(text):
  inputs = ["summarize: " + text]
  inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
  output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

  print(predicted_title)
  return predicted_title

summ_result = []
for txt in df['text']:
  summ_result.append(summarization(txt))

df['summarize'] = summ_result
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


개발Hadoop을 배우게 되면 MapReduce 다음으로 Yarn의 개념을 많이 접하게 되는데 대부분은 리소스 매니저라는 단어 정도로만 머릿속에 넣고 지나간다.
하둡하둡은 HDFS, MapReduce로 구성되며 개발자는 맵리듀스 프로그래밍 모델에 맞게 애플리케이션을 구현한다.
Opensearch는 Elastic Search가 기업화를 꾀하는데 대한 반발로 나온 오픈소스이며 분산형 커뮤니티 기반 100% 오픈 소스 검색 및 분석 제품군으로 다양한 사용 사례에 사용
10분동안 할 게 뭐 있다고 아둥바둥 기록하나 싶다가도 나한테 올바른 것이라면 손해보는 짓이라고 착각하지 말기나쁜짓만 아니면 아무것도 안하는 것보다 낫다 기억하기주말에는 4시간만 해보자.
Apache Airflow 소개 Apache Airflow 실습하며 배워보기 설치하고 실행하기 DAG 작성하기 Apache Airflow 실습하며 배워보기 설치하고 실행하기 DAG 작성하기
카프카 클러스터에 대한 대부분의 메타 데이터를 GET 요청 방식으로 조회할 수 있으며 새로운 Producer 설정을 전달하여 전역적으로 설정을 조정할 수 있다.
아파치 HBase는 하둡의 분산 파일 시스템인 HDFS위에서 동작하며 대량의 흩어져 있는 데이터 저장을 위해 구글의 BigTable과 비슷한 기능을 한다.
하둡 상에서 SQL을 이용해 MapReduce를 처리할 수 있는 하이브가 탄생하게 되었다.
FTP는 호스트 간에 두 가지 연결을 설정한다는 점에서 다른 클라이언트-서버 응용 프로그램과 다르며 하나의 연결은 데이터 전송에 사용되고 다른 하나는 제어 정보에 사용된다
System Programming 연재 마지막 포스팅은 Virtual Memory이다.
msgget은 메시지 큐의 식별자이며 전송 데이터는 long 값을 첫번째에 가지고 있어 원하는 데이터만 걸러낼 수 있다.
컴퓨터 내에서도 희생을 하지 않고 자원을 가지고 잇으면서 상대방이 요청한 자원을 가지고 있으면서 내가 가지고 있는 자원을 요청한다면 서로 원하는 자원을 영원

,Title,thumbnail,text,keyword,summarize
0,Hadoop Yarn 아키텍쳐,https://velog.velcdn.com/images/matteblack9/po...,"개발Hadoop을 배우게 되면, MapReduce 다음으로 Yarn의 개념을 가장 ...","[스케줄링 로부터, 노드 매니저, 문제 본질]",개발Hadoop을 배우게 되면 MapReduce 다음으로 Yarn의 개념을 많이 접...
1,하둡 프로그래밍 - MapReduce,https://velog.velcdn.com/images%2Fspdlqjfire%2...,"하둡하둡은 기본적으로 HDFS, MapReduce로 구성된다. 맵리듀스는 HDFS에...","[파일 분산, 개발자 분석, 적용 알고리즘]","하둡하둡은 HDFS, MapReduce로 구성되며 개발자는 맵리듀스 프로그래밍 모델..."
2,OpenSearch에 대해 알아보자!,https://velog.velcdn.com/images/sosimina/post/...,Opensearch는 Elastic Search가 기업화를 꾀하는데에 대한 반발로 ...,"[라이센스 무료, 대시보드 라파, 검색 분석]",Opensearch는 Elastic Search가 기업화를 꾀하는데 대한 반발로 나...
3,{일상} 10분만 일상,https://velog.velcdn.com/images/pairkorean/pro...,"{일상}10분동안 할게 뭐 있다고, 아둥바둥 힘들게 기록하나 싶다가도나한테 올바른 ...","[생태계 구축, 일요일, 서든 착각]",10분동안 할 게 뭐 있다고 아둥바둥 기록하나 싶다가도 나한테 올바른 것이라면 손해...
4,Airflow,https://images.velog.io/images/ifelifelse/post...,강의정리 - MLOps목차\nApache Airflow 소개\n-- Batch Pr...,"[소개 실습, 소프트웨어 자료구조, 에어비앤비 개발]",Apache Airflow 소개 Apache Airflow 실습하며 배워보기 설치하...


In [49]:
print('title :', df.loc[21,'Title'])
print('keyword :', df.loc[21,'keyword'])
print('summarize :', df.loc[21,'summarize'])

title : 동적 계획법(Dynamic Programming)
keyword : ['문제 수식', '코드 작성', '재귀 알고리즘']
summarize : DP는 큰 문제를 작은 문제로 나누어서 푸는 방식의 알고리즘으로 분할 정복과 가장 큰 차이점은 분할 정복은 쪼개진 작은 문제가 중복되지만 분할 정복은 절대로 중복될 수 없다는 점이다.


In [41]:
df.to_csv('KeywordSumm.csv', index=False)